In [ ]:
import warnings
warnings.filterwarnings("ignore")

# example of a dcgan on mnist
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import vstack
from numpy.random import randn
from numpy.random import randint
from keras.datasets.mnist import load_data
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from matplotlib import pyplot
import os


# creating the standalone discriminator model
def make_discriminator(in_shape=(28,28,1)):
	model = Sequential()
	# normal
	model.add(Conv2D(56, (3,3), padding='same', input_shape=in_shape))
	model.add(LeakyReLU(alpha=0.2))
	# downsample
	model.add(Conv2D(112, (3,3), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# downsample
	# model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
	# model.add(LeakyReLU(alpha=0.2))
	# downsample
	model.add(Conv2D(224, (3,3), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# classifier
	model.add(Flatten())
	model.add(Dropout(0.4))
	model.add(Dense(1, activation='sigmoid'))
	# compile model
	learning_rate = 0.0002
	beta1_val = 0.5
	opt = Adam(lr = learning_rate, beta_1 = beta1_val)
	model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
	print(model.summary())
	return model

# creating the standalone generator model
def make_generator(latent_dim):
	model = Sequential()
	# Downsample the following 2 variables by 2 if layers are added
	h = 14
	w = 14
	c = 256
	num_nodes = c*h*w
	model.add(Dense(num_nodes, input_dim=latent_dim))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Reshape((h, w, c)))
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# Following layers can be uncommented for a deeper network, please downsample h,w by two for each layer added
	# model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	# model.add(LeakyReLU(alpha=0.2))
	# output layer
	model.add(Conv2D(1, (3,3), activation='tanh', padding='same'))
	print(model.summary())
	return model


def define_gan(generator_model, disc_model):

	disc_model.trainable = False

	model = Sequential()

	model.add(generator_model)

	model.add(disc_model)

	learning_rate = 0.0002
	beta1_val = 0.5
	opt = Adam(lr = learning_rate, beta_1 = beta1_val)
	model.compile(loss='binary_crossentropy', optimizer=opt)
	return model

# loading and transforming data for training
def load_real_samples():

	(trainX,_), (_,_) = load_data()

	X = trainX.astype('float32')

	X = (X - 127.5) / 127.5
	return X

# select real samples
def create_real_samples(dataset, n_samples):
	# choose random instances
	ix = randint(0, dataset.shape[0], n_samples)

	X = dataset[ix]

	y = ones((n_samples, 1))
	return X, y


def gen_latent_points(latent_dim, n_samples):

	x_input = randn(latent_dim * n_samples)

	x_input = x_input.reshape(n_samples, latent_dim)
	return x_input


def create_fake_samples(generator_model, latent_dim, n_samples):
	# generate points in latent space
	x_input = gen_latent_points(latent_dim, n_samples)

	X = generator_model.predict(x_input)

	y = zeros((n_samples, 1))
	return X, y

# function to save generated images after each epoch
def save_plot(examples, epoch, n=7, output_dir='generated_images'):

    os.makedirs(output_dir, exist_ok=True)

    examples = (examples + 1) / 2.0

    for i in range(n * n):
        grayscale_image = examples[i]

        pyplot.subplot(n, n, 1 + i)
        pyplot.axis('off')
        pyplot.imshow(grayscale_image, cmap='gray')  # Use 'gray' colormap for grayscale

    filename = os.path.join(output_dir, 'generated_images%03d.png' % (epoch + 1))
    pyplot.savefig(filename)
    pyplot.close()

# function to output the performance of the model and save the model after each epoch
def print_performance(epoch, generator_model, disc_model, dataset, latent_dim, n_samples=150,output_dir='generatedisc_models'):
	os.makedirs(output_dir, exist_ok=True)

	X_real, y_real = create_real_samples(dataset, n_samples)

	_, real_acc = disc_model.evaluate(X_real, y_real, verbose=0)

	x_fake, y_fake = create_fake_samples(generator_model, latent_dim, n_samples)

	_, fake_acc = disc_model.evaluate(x_fake, y_fake, verbose=0)

	print('>Accuracy real: %.0f%%, fake: %.0f%%' % (real_acc*100, fake_acc*100))

	save_plot(x_fake, epoch)

	filename = os.path.join(output_dir, 'generated_model%03d' % (epoch + 1))
	generator_model.save(filename)

# train the generator and discriminator
def train(generator_model, disc_model, gan_model, dataset, latent_dim, num_epochs, batch_size):
	num_of_batches = int(dataset.shape[0] / batch_size)
	half_batch = int(batch_size / 2)

	for i in range(num_epochs):

		for j in range(num_of_batches):

			X_real, y_real = create_real_samples(dataset, half_batch)

			d_loss1, _ = disc_model.train_on_batch(X_real, y_real)

			X_fake, y_fake = create_fake_samples(generator_model, latent_dim, half_batch)

			d_loss2, _ = disc_model.train_on_batch(X_fake, y_fake)

			X_gan = gen_latent_points(latent_dim, batch_size)

			y_gan = ones((batch_size, 1))
			g_loss = gan_model.train_on_batch(X_gan, y_gan)

			print('>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' %
				(i+1, j+1, num_of_batches, d_loss1, d_loss2, g_loss))

		print_performance(i, generator_model, disc_model, dataset, latent_dim)


latent_dim = 100
disc_model = make_discriminator()
generator_model = make_generator(latent_dim)
gan_model = define_gan(generator_model, disc_model)
dataset = load_real_samples()
num_epochs = 100
batch_size = 128
train(generator_model, disc_model, gan_model, dataset, latent_dim, num_epochs, batch_size)

In [ ]:
# example of loading the generator model and generating images
from keras.models import load_model
from numpy.random import randn
from matplotlib import pyplot

# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
	# generate points in the latent space
	x_input = randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	x_input = x_input.reshape(n_samples, latent_dim)
	return x_input

# plot the generated images
def create_plot(examples, n):
	# plot images
	for i in range(n * n):
		# define subplot
		pyplot.subplot(n, n, 1 + i)
		# turn off axis
		pyplot.axis('off')
		# plot raw pixel data
		pyplot.imshow(examples[i, :, :])
	pyplot.show()

# load model
model = load_model('paste the path where model is stored')
# generate images
latent_points = generate_latent_points(100, 100)
# generate images
X = model.predict(latent_points)
# scale from [-1,1] to [0,1]
X = (X + 1) / 2.0
# plot the result
create_plot(X, 10)